In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
import subprocess
import sys

#Import utils functions
curr_dir = !pwd
sys.path.append(curr_dir[0]+"/../utils")
from prediction_general_funcs import ligands

In [12]:
curr_dir = !pwd
datafile_date = "08.06.18"
prec_th = 0.5
folds_num = 5
input_path = curr_dir[0]+"/mediode_NegLigand_NoFilter/NoDown/"+datafile_date+"_"+str(folds_num)+"f_comb_dna0.5_rna0.25_ion0.75/per_fold/"
output_path = curr_dir[0]+"/mediode_NegLigand_NoFilter/NoDown/"+datafile_date+"_"+str(folds_num)+"f_comb_dna0.5_rna0.25_ion0.75/"
_
save_files = True

# input_path = curr_dir[0]+"/mediode_NegLigand_NoFilter/NoDown/"+datafile_date+"_"+str(folds_num)+"f_0.5p/per_fold/"
# output_path = curr_dir[0]+"/mediode_NegLigand_NoFilter/NoDown/"+datafile_date+"_"+str(folds_num)+"f_0.5p/"

ligands = ["dna", "rna", "ion", "peptide", "sm"]

#classifiers = ["XGB", "RF", "SVM", "KNN", "ADA", "Logistic", "NN"]
classifiers = ["XGB", "SVM", "RF", "Logistic", "ADA", "KNN"]
#versions = ["10","10w","5","5w","3","3w"]
version = str(folds_num)+"w"

#### Copy relevant files from gen-comp1

In [21]:
!scp -r -p anatf@gen-comp1:/Genomics/grid/users/anatf/ExAC/10.Prediction/pred_AUC_AUPRC/mediode_NegLigand_NoFilter/NoDown/06.20.18_5f_comb_dna0.5_rna0.25_ion0.75_tuned/per_fold/* /home/anat/Research/ExAC/10.Prediction/pred_AUC_AUPRC/mediode_NegLigand_NoFilter/NoDown/06.20.18_5f_comb_dna0.5_rna0.25_ion0.75_tuned/per_fold/.

dna_ADA_fold1_5w_auc.csv                      100%   26     0.0KB/s   00:00    
dna_ADA_fold1_5w_auprc.csv                    100%   27     0.0KB/s   00:00    
dna_ADA_fold1_5w.csv                          100%  339KB 339.2KB/s   00:00    
dna_ADA_fold1_5w_d_auc.csv                    100%  248     0.2KB/s   00:00    
dna_ADA_fold1_5w_d_auprc.csv                  100%  261     0.3KB/s   00:00    
dna_ADA_fold1_5w_d_auprc_ratio.csv            100%  260     0.3KB/s   00:00    
dna_ADA_fold1_5w_d.csv                        100%   11KB  11.3KB/s   00:00    
dna_ADA_fold1_5w_dm_auc.csv                   100%   26     0.0KB/s   00:00    
dna_ADA_fold1_5w_dm_auprc.csv                 100%   26     0.0KB/s   00:00    
dna_ADA_fold1_5w_dm_auprc_ratio.csv           100%   26     0.0KB/s   00:00    
dna_ADA_fold2_5w_auc.csv                      100%   26     0.0KB/s   00:00    
dna_ADA_fold2_5w_auprc.csv                    100%   26     0.0KB/s   00:00    
dna_ADA_fold2_5w.csv                    

dna_Logistic_fold1_5w_d_auc.csv               100%  251     0.3KB/s   00:00    
dna_Logistic_fold1_5w_d_auprc.csv             100%  271     0.3KB/s   00:00    
dna_Logistic_fold1_5w_d_auprc_ratio.csv       100%  265     0.3KB/s   00:00    
dna_Logistic_fold1_5w_d.csv                   100%   13KB  13.0KB/s   00:00    
dna_Logistic_fold1_5w_dm_auc.csv              100%   31     0.0KB/s   00:00    
dna_Logistic_fold1_5w_dm_auprc.csv            100%   31     0.0KB/s   00:00    
dna_Logistic_fold1_5w_dm_auprc_ratio.csv      100%   31     0.0KB/s   00:00    
dna_Logistic_fold2_5w_auc.csv                 100%   31     0.0KB/s   00:00    
dna_Logistic_fold2_5w_auprc.csv               100%   31     0.0KB/s   00:00    
dna_Logistic_fold2_5w.csv                     100%  415KB 415.5KB/s   00:00    
dna_Logistic_fold2_5w_d_auc.csv               100%  317     0.3KB/s   00:00    
dna_Logistic_fold2_5w_d_auprc.csv             100%  322     0.3KB/s   00:00    
dna_Logistic_fold2_5w_d_auprc_ratio.csv 

dna_SVM_fold1_5w_d.csv                        100%   12KB  11.8KB/s   00:00    
dna_SVM_fold1_5w_dm_auc.csv                   100%   26     0.0KB/s   00:00    
dna_SVM_fold1_5w_dm_auprc.csv                 100%   26     0.0KB/s   00:00    
dna_SVM_fold1_5w_dm_auprc_ratio.csv           100%   26     0.0KB/s   00:00    
dna_SVM_fold2_5w_auc.csv                      100%   26     0.0KB/s   00:00    
dna_SVM_fold2_5w_auprc.csv                    100%   27     0.0KB/s   00:00    
dna_SVM_fold2_5w.csv                          100%  379KB 378.5KB/s   00:00    
dna_SVM_fold2_5w_d_auc.csv                    100%  301     0.3KB/s   00:00    
dna_SVM_fold2_5w_d_auprc.csv                  100%  319     0.3KB/s   00:00    
dna_SVM_fold2_5w_d_auprc_ratio.csv            100%  310     0.3KB/s   00:00    
dna_SVM_fold2_5w_d.csv                        100%   30KB  30.5KB/s   00:00    
dna_SVM_fold2_5w_dm_auc.csv                   100%   26     0.0KB/s   00:00    
dna_SVM_fold2_5w_dm_auprc.csv           

ion_ADA_fold2_5w.csv                          100%  325KB 325.4KB/s   00:00    
ion_ADA_fold2_5w_d_auc.csv                    100%  456     0.5KB/s   00:00    
ion_ADA_fold2_5w_d_auprc.csv                  100%  448     0.4KB/s   00:00    
ion_ADA_fold2_5w_d_auprc_ratio.csv            100%  424     0.4KB/s   00:00    
ion_ADA_fold2_5w_d.csv                        100%   69KB  68.7KB/s   00:00    
ion_ADA_fold2_5w_dm_auc.csv                   100%   26     0.0KB/s   00:00    
ion_ADA_fold2_5w_dm_auprc.csv                 100%   26     0.0KB/s   00:00    
ion_ADA_fold2_5w_dm_auprc_ratio.csv           100%   26     0.0KB/s   00:00    
ion_ADA_fold3_5w_auc.csv                      100%   26     0.0KB/s   00:00    
ion_ADA_fold3_5w_auprc.csv                    100%   27     0.0KB/s   00:00    
ion_ADA_fold3_5w.csv                          100%  339KB 339.5KB/s   00:01    
ion_ADA_fold3_5w_d_auc.csv                    100%  765     0.8KB/s   00:00    
ion_ADA_fold3_5w_d_auprc.csv            

ion_Logistic_fold2_5w_d_auprc_ratio.csv       100%  434     0.4KB/s   00:00    
ion_Logistic_fold2_5w_d.csv                   100%   81KB  80.7KB/s   00:00    
ion_Logistic_fold2_5w_dm_auc.csv              100%   31     0.0KB/s   00:00    
ion_Logistic_fold2_5w_dm_auprc.csv            100%   31     0.0KB/s   00:00    
ion_Logistic_fold2_5w_dm_auprc_ratio.csv      100%   31     0.0KB/s   00:00    
ion_Logistic_fold3_5w_auc.csv                 100%   31     0.0KB/s   00:00    
ion_Logistic_fold3_5w_auprc.csv               100%   31     0.0KB/s   00:00    
ion_Logistic_fold3_5w.csv                     100%  393KB 393.4KB/s   00:00    
ion_Logistic_fold3_5w_d_auc.csv               100%  756     0.7KB/s   00:00    
ion_Logistic_fold3_5w_d_auprc.csv             100%  780     0.8KB/s   00:00    
ion_Logistic_fold3_5w_d_auprc_ratio.csv       100%  787     0.8KB/s   00:00    
ion_Logistic_fold3_5w_d.csv                   100%  127KB 127.0KB/s   00:00    
ion_Logistic_fold3_5w_dm_auc.csv        

...

rna_RF_fold3_5w_d_auprc.csv                   100%  225     0.2KB/s   00:00    
rna_RF_fold3_5w_d_auprc_ratio.csv             100%  223     0.2KB/s   00:00    
rna_RF_fold3_5w_d.csv                         100% 9705     9.5KB/s   00:00    
rna_RF_fold3_5w_dm_auc.csv                    100%   24     0.0KB/s   00:00    
rna_RF_fold3_5w_dm_auprc.csv                  100%   25     0.0KB/s   00:00    
rna_RF_fold3_5w_dm_auprc_ratio.csv            100%   24     0.0KB/s   00:00    
rna_RF_fold4_5w_auc.csv                       100%   25     0.0KB/s   00:00    
rna_RF_fold4_5w_auprc.csv                     100%   26     0.0KB/s   00:00    
rna_RF_fold4_5w.csv                           100%  395KB 394.9KB/s   00:00    
rna_RF_fold4_5w_d_auc.csv                     100%  127     0.1KB/s   00:00    
rna_RF_fold4_5w_d_auprc.csv                   100%  128     0.1KB/s   00:00    
rna_RF_fold4_5w_d_auprc_ratio.csv             100%  126     0.1KB/s   00:00    
rna_RF_fold4_5w_d.csv                   

rna_XGB_fold3_5w_dm_auc.csv                   100%   26     0.0KB/s   00:00    
rna_XGB_fold3_5w_dm_auprc.csv                 100%   26     0.0KB/s   00:00    
rna_XGB_fold3_5w_dm_auprc_ratio.csv           100%   24     0.0KB/s   00:00    
rna_XGB_fold4_5w_auc.csv                      100%   24     0.0KB/s   00:00    
rna_XGB_fold4_5w_auprc.csv                    100%   27     0.0KB/s   00:00    
rna_XGB_fold4_5w.csv                          100%  401KB 400.8KB/s   00:00    
rna_XGB_fold4_5w_d_auc.csv                    100%  113     0.1KB/s   00:00    
rna_XGB_fold4_5w_d_auprc.csv                  100%  127     0.1KB/s   00:00    
rna_XGB_fold4_5w_d_auprc_ratio.csv            100%  111     0.1KB/s   00:00    
rna_XGB_fold4_5w_d.csv                        100%   16KB  15.6KB/s   00:00    
rna_XGB_fold4_5w_dm_auc.csv                   100%   27     0.0KB/s   00:00    
rna_XGB_fold4_5w_dm_auprc.csv                 100%   27     0.0KB/s   00:00    
rna_XGB_fold4_5w_dm_auprc_ratio.csv     

sm_KNN_fold4_5w.csv                           100%  250KB 250.3KB/s   00:00    
sm_KNN_fold4_5w_d_auc.csv                     100%  883     0.9KB/s   00:00    
sm_KNN_fold4_5w_d_auprc.csv                   100%  889     0.9KB/s   00:00    
sm_KNN_fold4_5w_d_auprc_ratio.csv             100%  857     0.8KB/s   00:00    
sm_KNN_fold4_5w_d.csv                         100%  100KB 100.3KB/s   00:00    
sm_KNN_fold4_5w_dm_auc.csv                    100%   26     0.0KB/s   00:00    
sm_KNN_fold4_5w_dm_auprc.csv                  100%   26     0.0KB/s   00:00    
sm_KNN_fold4_5w_dm_auprc_ratio.csv            100%   25     0.0KB/s   00:00    
sm_KNN_fold5_5w_auc.csv                       100%   26     0.0KB/s   00:00    
sm_KNN_fold5_5w_auprc.csv                     100%   27     0.0KB/s   00:00    
sm_KNN_fold5_5w.csv                           100%  240KB 239.6KB/s   00:00    
sm_KNN_fold5_5w_d_auc.csv                     100%  766     0.8KB/s   00:00    
sm_KNN_fold5_5w_d_auprc.csv             

sm_RF_fold5_5w_auc.csv                        100%   25     0.0KB/s   00:00    
sm_RF_fold5_5w_auprc.csv                      100%   26     0.0KB/s   00:00    
sm_RF_fold5_5w.csv                            100%  260KB 260.3KB/s   00:00    
sm_RF_fold5_5w_d_auc.csv                      100%  752     0.7KB/s   00:00    
sm_RF_fold5_5w_d_auprc.csv                    100%  739     0.7KB/s   00:00    
sm_RF_fold5_5w_d_auprc_ratio.csv              100%  742     0.7KB/s   00:00    
sm_RF_fold5_5w_d.csv                          100%  138KB 138.3KB/s   00:00    
sm_RF_fold5_5w_dm_auc.csv                     100%   25     0.0KB/s   00:00    
sm_RF_fold5_5w_dm_auprc.csv                   100%   26     0.0KB/s   00:00    
sm_RF_fold5_5w_dm_auprc_ratio.csv             100%   24     0.0KB/s   00:00    
sm_SVM_fold1_5w_auc.csv                       100%   26     0.0KB/s   00:00    
sm_SVM_fold1_5w_auprc.csv                     100%   25     0.0KB/s   00:00    
sm_SVM_fold1_5w.csv                     

sm_XGB_fold5_5w_d_auprc.csv                   100%  791     0.8KB/s   00:00    
sm_XGB_fold5_5w_d_auprc_ratio.csv             100%  612     0.6KB/s   00:00    
sm_XGB_fold5_5w_d.csv                         100%  142KB 142.4KB/s   00:00    
sm_XGB_fold5_5w_dm_auc.csv                    100%   26     0.0KB/s   00:00    
sm_XGB_fold5_5w_dm_auprc.csv                  100%   26     0.0KB/s   00:00    
sm_XGB_fold5_5w_dm_auprc_ratio.csv            100%   26     0.0KB/s   00:00    


In [13]:
for ligand in ligands:
    
    ### Global Metrics ###
    #Combine AUC folds
    auc_df = pd.DataFrame()
    for classifier in classifiers:
        auc_list = []
        for fold in range(1,(folds_num+1)):
            auc_fold_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_"+version+"_auc.csv"
            curr_auc_df = pd.read_csv(auc_fold_filename, sep='\t', index_col=0)
            auc_list.append(float(curr_auc_df.iloc[0]))
        auc_df[classifier] = auc_list
        print ligand+" "+classifier+" AUC avg:"+str(np.mean(auc_list))
    #Write AUC table to file
    auc_df.to_csv(output_path+ligand+"_"+version+"_auc.csv", sep='\t')
    
    #Combine AUPRC folds
    auprc_df = pd.DataFrame()
    for classifier in classifiers:
        auprc_list = []
        for fold in range(1,(folds_num+1)):
            auprc_fold_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_"+version+"_auprc.csv"
            curr_auprc_df = pd.read_csv(auprc_fold_filename, sep='\t', index_col=0)
            auprc_list.append(float(curr_auprc_df.iloc[0]))
        auprc_df[classifier] = auprc_list
        print ligand+" "+classifier+" AUPRC avg:"+str(np.mean(auprc_list))
    #Write AUPRC table to file
    if (save_files): auprc_df.to_csv(output_path+ligand+"_"+version+"_auprc.csv", sep='\t')
    
    ### Per-Domain Metrics ###
    #Combine domain mean AUC folds
    domain_auc_mean_df = pd.DataFrame()
    for classifier in classifiers:
        domain_auc_mean_list = []
        for fold in range(1,(folds_num+1)):
            domain_auc_mean_fold_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_"+version+"_dm_auc.csv"
            curr_domain_auc_mean_df = pd.read_csv(domain_auc_mean_fold_filename, sep='\t', index_col=0)
            domain_auc_mean_list.append(float(curr_domain_auc_mean_df.iloc[0]))
        domain_auc_mean_df[classifier] = domain_auc_mean_list
        print ligand+" "+classifier+" domain AUC mean avg:"+str(np.mean(domain_auc_mean_list))
    #Write AUC table to file
    if (save_files): domain_auc_mean_df.to_csv(output_path+ligand+"_"+version+"_dm_auc.csv", sep='\t')
    
    #Combine domain mean AUPRC folds
    domain_auprc_mean_df = pd.DataFrame()
    for classifier in classifiers:
        domain_auprc_mean_list = []
        for fold in range(1,(folds_num+1)):
            domain_auprc_mean_fold_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_"+version+"_dm_auprc.csv"
            curr_domain_auprc_mean_df = pd.read_csv(domain_auprc_mean_fold_filename, sep='\t', index_col=0)
            domain_auprc_mean_list.append(float(curr_domain_auprc_mean_df.iloc[0]))
        domain_auprc_mean_df[classifier] = domain_auprc_mean_list
        print ligand+" "+classifier+" domain AUPRC mean avg:"+str(np.mean(domain_auprc_mean_list))
    #Write AUPRC table to file
    if (save_files): domain_auprc_mean_df.to_csv(output_path+ligand+"_"+version+"_dm_auprc.csv", sep='\t')
    
    #Combine domain mean AUPRC ratio folds
    domain_auprc_ratio_mean_df = pd.DataFrame()
    for classifier in classifiers:
        domain_auprc_ratio_mean_list = []
        for fold in range(1,(folds_num+1)):
            domain_auprc_ratio_mean_fold_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_"+version+"_dm_auprc_ratio.csv"
            curr_domain_auprc_ratio_mean_df = pd.read_csv(domain_auprc_ratio_mean_fold_filename, sep='\t', index_col=0)
            domain_auprc_ratio_mean_list.append(float(curr_domain_auprc_ratio_mean_df.iloc[0]))
        domain_auprc_ratio_mean_df[classifier] = domain_auprc_ratio_mean_list
        print ligand+" "+classifier+" domain AUPRC ratio mean avg:"+str(np.mean(domain_auprc_ratio_mean_list))
    #Write AUPRC table to file
    if (save_files): domain_auprc_ratio_mean_df.to_csv(output_path+ligand+"_"+version+"_dm_auprc_ratio.csv", sep='\t')
    
    #Combine domain (not mean) AUCs folds
    domain_auc_df = pd.DataFrame()
    for classifier in classifiers:
        domain_auc_list = []
        domain_name_list = []
        pos_num_list = []
        neg_num_list = []
        update_domain_params = True
        for fold in range(1,(folds_num+1)):
            domain_auc_fold_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_"+version+"_d_auc.csv"
            curr_domain_auc_df = pd.read_csv(domain_auc_fold_filename, sep='\t', index_col=0)
            domain_auc_list.extend([float(x) for x in curr_domain_auc_df[classifier]])
            if (update_domain_params):
                domain_name_list.extend(curr_domain_auc_df["domain"])
                pos_num_list.extend(curr_domain_auc_df["pos_num"])
                neg_num_list.extend(curr_domain_auc_df["neg_num"])
        domain_auc_df[classifier] = domain_auc_list
        print ligand+" "+classifier+" domains AUCs avg:"+str(np.mean(domain_auc_list))
        update_domain_params = False
    domain_auc_df["domain"] = domain_name_list
    domain_auc_df["pos_num"] = pos_num_list
    domain_auc_df["neg_num"] = neg_num_list
    #Write AUC table to file
    if (save_files): domain_auc_df.to_csv(output_path+ligand+"_"+version+"_d_auc.csv", sep='\t')
    
    #Combine domain (not mean) AUPRCs folds
    domain_auprc_df = pd.DataFrame()
    for classifier in classifiers:
        domain_auprc_list = []
        domain_name_list = []
        pos_num_list = []
        neg_num_list = []
        update_domain_params = True
        for fold in range(1,(folds_num+1)):
            domain_auprc_fold_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_"+version+"_d_auprc.csv"
            curr_domain_auprc_df = pd.read_csv(domain_auprc_fold_filename, sep='\t', index_col=0)
            domain_auprc_list.extend([float(x) for x in curr_domain_auprc_df[classifier]])
            if (update_domain_params):
                domain_name_list.extend(curr_domain_auprc_df["domain"])
                pos_num_list.extend(curr_domain_auprc_df["pos_num"])
                neg_num_list.extend(curr_domain_auprc_df["neg_num"])
        domain_auprc_df[classifier] = domain_auprc_list
        print ligand+" "+classifier+" domains AUPRCs avg:"+str(np.mean(domain_auprc_list))
        update_domain_params = False
    domain_auprc_df["domain"] = domain_name_list
    domain_auprc_df["pos_num"] = pos_num_list
    domain_auprc_df["neg_num"] = neg_num_list
    #Write AUPRC table to file
    if (save_files): domain_auprc_df.to_csv(output_path+ligand+"_"+version+"_d_auprc.csv", sep='\t')
    
    #Combine domain (not mean) AUPRCs ratio folds
    domain_auprc_ratio_df = pd.DataFrame()
    for classifier in classifiers:
        domain_auprc_ratio_list = []
        domain_name_list = []
        pos_num_list = []
        neg_num_list = []
        update_domain_params = True
        for fold in range(1,(folds_num+1)):
            domain_auprc_ratio_fold_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_"+version+"_d_auprc_ratio.csv"
            curr_domain_auprc_ratio_df = pd.read_csv(domain_auprc_ratio_fold_filename, sep='\t', index_col=0)
            domain_auprc_ratio_list.extend([float(x) for x in curr_domain_auprc_ratio_df[classifier]])
            if (update_domain_params):
                domain_name_list.extend(curr_domain_auprc_ratio_df["domain"])
                pos_num_list.extend(curr_domain_auprc_ratio_df["pos_num"])
                neg_num_list.extend(curr_domain_auprc_ratio_df["neg_num"])
        domain_auprc_ratio_df[classifier] = domain_auprc_ratio_list
        print ligand+" "+classifier+" domains AUPRCs ratio avg:"+str(np.mean(domain_auprc_ratio_list))
        update_domain_params = False
    domain_auprc_ratio_df["domain"] = domain_name_list
    domain_auprc_ratio_df["pos_num"] = pos_num_list
    domain_auprc_ratio_df["neg_num"] = neg_num_list
    #Write AUPRC table to file
    if (save_files): domain_auprc_ratio_df.to_csv(output_path+ligand+"_"+version+"_d_auprc_ratio.csv", sep='\t')
    
    #Combining global probs results files
    probs_df = pd.DataFrame()
    for classifier in classifiers:
        for fold in range(1,(folds_num+1)):
            probs_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_"+version+".csv"
            curr_probs_df = pd.read_csv(probs_filename, sep='\t', index_col=0)
            if (len(probs_df.columns) == 0):
                probs_df = curr_probs_df
            else:
                probs_df = probs_df.append(curr_probs_df, ignore_index=True)
    #Write probs table to file 
    probs_df.to_csv(output_path+ligand+"_"+version+".csv", sep='\t')
    
    #Combining global probs results files
    domain_probs_df = pd.DataFrame()
    for classifier in classifiers:
        for fold in range(1,(folds_num+1)):
            domain_probs_filename = input_path+ligand+"_"+classifier+"_fold"+str(fold)+"_"+version+"_d.csv"
            curr_domain_probs_df = pd.read_csv(domain_probs_filename, sep='\t', index_col=0)
            if (len(domain_probs_df.columns) == 0):
                domain_probs_df = curr_domain_probs_df
            else:
                domain_probs_df = domain_probs_df.append(curr_domain_probs_df, ignore_index=True)
    #Write probs table to file 
    if (save_files): domain_probs_df.to_csv(output_path+ligand+"_"+version+"_d.csv", sep='\t')

    print "Finished ligand "+ligand 

dna XGB AUC avg:0.9101511222543245
dna SVM AUC avg:0.8923846836731094
dna RF AUC avg:0.8884134875621499
dna Logistic AUC avg:0.8834831587504824
dna ADA AUC avg:0.8805031445419154
dna KNN AUC avg:0.8223910393260256
dna XGB AUPRC avg:0.33171155616425063
dna SVM AUPRC avg:0.2846106757653809
dna RF AUPRC avg:0.33451270298663915
dna Logistic AUPRC avg:0.3071403093599133
dna ADA AUPRC avg:0.26332538672969036
dna KNN AUPRC avg:0.24867297184997178
dna XGB domain AUC mean avg:0.7836803472076539
dna SVM domain AUC mean avg:0.7287726356993972
dna RF domain AUC mean avg:0.7030128342353839
dna Logistic domain AUC mean avg:0.7383182668354804
dna ADA domain AUC mean avg:0.7246118211914814
dna KNN domain AUC mean avg:0.6756793812748748
dna XGB domain AUPRC mean avg:0.5525657179777119
dna SVM domain AUPRC mean avg:0.5198419553818729
dna RF domain AUPRC mean avg:0.49719458369741804
dna Logistic domain AUPRC mean avg:0.5208259335669597
dna ADA domain AUPRC mean avg:0.4909953396985623
dna KNN domain AUPRC

peptide KNN domain AUPRC ratio mean avg:2.659722606013093
peptide XGB domains AUCs avg:0.7537669548714477
peptide SVM domains AUCs avg:0.5644499230052653
peptide RF domains AUCs avg:0.7107622236672495
peptide Logistic domains AUCs avg:0.7056203825464048
peptide ADA domains AUCs avg:0.6255898785442439
peptide KNN domains AUCs avg:0.6181826612132912
peptide XGB domains AUPRCs avg:0.34035308898879957
peptide SVM domains AUPRCs avg:0.1849102454738607
peptide RF domains AUPRCs avg:0.2819452835839587
peptide Logistic domains AUPRCs avg:0.2803780075026218
peptide ADA domains AUPRCs avg:0.3253347763742493
peptide KNN domains AUPRCs avg:0.250729979212877
peptide XGB domains AUPRCs ratio avg:4.998106379722688
peptide SVM domains AUPRCs ratio avg:1.8414243867891444
peptide RF domains AUPRCs ratio avg:4.005115168320163
peptide Logistic domains AUPRCs ratio avg:3.255792152351056
peptide ADA domains AUPRCs ratio avg:6.13579527698987
peptide KNN domains AUPRCs ratio avg:2.7391082681485184
Finished li